In [1]:
import torch
import math
torch.set_grad_enabled(False)

In [2]:
def generate_set():
    training_set = torch.empty(1000,2).uniform_(0,1)  #x et y
    training_classes = torch.empty(1000)
    testing_set = torch.empty(1000,2).uniform_(0,1)  #x et y
    testing_classes = torch.empty(1000)

    r = torch.empty(1,1).fill_(1/(2*math.pi)).pow(1/2)

    for i in range (1000):
        if ((training_set[i] - torch.Tensor([0.5,0.5])).pow(2).sum()).pow(1/2).item() < r.item():
            training_classes[i] = 1
        else:
            training_classes[i] = 0

        if ((testing_set[i] - torch.Tensor([0.5,0.5])).pow(2).sum()).pow(1/2).item() < r.item():
            testing_classes[i] = 1
        else:
            testing_classes[i] = 0
    return training_set, training_classes, testing_set, testing_classes

In [3]:
training_set, training_classes, testing_set, testing_classes = generate_set()

In [4]:
import matplotlib.pyplot as plt
plt.figure('1')
x = training_set[:,0]
y = training_set[:,1]
plt.scatter(x[training_classes==1], y[training_classes==1], color='r')
plt.scatter(x[training_classes==0], y[training_classes==0], color='b')

print(sum(training_classes==0))
print(sum(training_classes==1))
plt.show()

tensor(504)
tensor(496)


<Figure size 640x480 with 1 Axes>

# LINEAR

In [5]:
#LINEAR MODULE (FULLY CONNECTED LAYER)
class Linear(object):
    def __init__(self, dimension, nb_data_in, nb_data_out):
        #x = nb_data_in x dim in my calculation, but here transpose (dim x nb_data_in)
        #y = nb_data_out x dim in my calculation, but here transpose (dim x nb_data_out)
        k = math.sqrt(1/nb_data_in)
        self.weight = torch.empty(nb_data_out,nb_data_in).uniform_(-k,k)
        self.bias = torch.empty(nb_data_out,dimension).uniform_(-k,k)
        self.grad_weight = None
        self.grad_bias = None
        self.input = None
        
    def updateparam(self, lr):
        for i in range(len(self.grad_weight)): 
            self.weight -= lr * self.grad_weight
            self.bias -= lr * self.grad_bias
    
    def forward(self , input, nb_layer):
        #All the calculation were done considering x = nb_data_in x dim (as seen in the lesson)
        #So to simplify the comprehension, we do a first transpose, do the the calculations with this
        #then transpose again after, to have the correct dimension output
       
        #save x with dim = dim x nb_data_in    
        self.input = input
        output = None
        
        #x = dim x nb_data_in -> nb_data_in x dim
        x_correct_dim = input.t()
        #y = nb_data_out x dim
        y_correct_dim = None

        #dim analysis: [nb_data_out x nb_data_in] x [nb_data_in x dim] + [nb_data_out x dim]
        y_correct_dim = ((self.weight).matmul(x_correct_dim)+self.bias)

        #append y = [dim x nb_data_out] to respect lin module 
        return (y_correct_dim.t())
    
    def backward(self, gradwrtoutput):
        dl_dx  = None
        dl_dw = None
        dl_db = None
        gradaccumulated = None
        
        #X = dim x nb_data_in -> nb_data_in x dim
        x_correct_dim = (self.input).t()

        #X = dim x nb_data_out -> nb_data_out x dim
        grad_correct_dim = gradwrtoutput.t()

        #dl_dx = w^T x dl_dy (gradwrtoutput)
        #dim analysis: nb_data_in x dim = [nb_data_in x nb_data_out] x [nb_data_out x dim]
        dl_dx = (self.weight).t().matmul(grad_correct_dim)
        #nb_data_in x dim -> dim x nb_data_in
        gradaccumulated = dl_dx.t()

        #dl_db = dl_dy
        #dim analysis: nb_data_out x dim (car b = nb_data_out x dim)
        self.grad_bias = grad_correct_dim

        #dl_dw = dl_dy x X^T
        #dim analysis: nb_data_out x nb_data_in = [nb_data_out x dim] x [dim x nb_data_in]
        self.grad_weight = grad_correct_dim.matmul(x_correct_dim.t())
            
        return gradaccumulated
        
    def param(self):
        output = [[self.weight, self.grad_weight], [self.bias, self.grad_bias]]
        return output
    
    #https://pytorch.org/docs/stable/nn.html#linear-layers

# RELU

In [6]:
#RELU MODULE
class ReLU():
    def __init__(self):
        self.input = None
    
    def forward(self, input):
        self.input = input
        return torch.max(input,torch.zeros_like(x))
        
    def backward(self, gradwrtoutput): 
        dx = ((self.input)>=0).float()
        return dx*gradwrtoutput 

    def param(self): 
        return []
    
#backward should get as input a tensor or a tuple of tensors containing the gradient of the 
#loss with respect to the module’s output, accumulate the gradient wrt the parameters, 
#and return a tensor or a tuple of tensors containing the gradient of the loss wrt the module’s input.

# TANH

In [7]:
#TANH MODULE
class Tanh():
    def __init__(self):
        self.input = None
        
    def forward(self, input):
        self.input = input
        return torch.tanh(input)
    
    def backward(self, gradwrtoutput):
        return (1 - torch.tanh(self.input).pow(2))*gradwrtoutput 

    def param(self):
        return [] #Pas de param

# LOSSMSE

In [8]:
#LOSSMSE MODULE
    
class LossMSE():
    
    def forward(self, input, target): 
        loss = torch.mean((input-target).pow(2))
        return loss
        
    def backward(self, input, target):
        target = target.unsqueeze(0)
        #print(type(input), input.size(), input, type(target),target.size(), target)
        dloss = (2*(input - target))/(input.size(1))
        return dloss

    def param(self):
        return [] #No param

In [9]:
def class_into_2(classes):
    t = torch.empty(classes.size(0),2).zero_()
    for n in range (classes.size(0)):
        t[n,int(classes[n].item())] = 1
    return t

# Gradient 2 layer Tanh

In [45]:
nb_iterations = 50
print(nb_iterations)

lin1 = Linear(1,2,25)
lin2 = Linear(1,25,2)
T1 = Tanh()
L = LossMSE()
  

for i in range (nb_iterations):
    print()
    print('iteration', i)
    loss_training = 0
    
    #1000 x 1 -> 1000 x 2:  [1 0] and [0 1]
    train_target = class_into_2(training_classes);
    test_target = class_into_2(testing_classes);
    
    #Etape Update valeur par valeur
    for j in range(training_set.size(0)):
        f1 = lin1.forward(training_set[j].unsqueeze(0), nb_layer = 1)
        #print(f1)
        t1 = T1.forward(f1)
        #print(t1)
        output = (lin2.forward(t1, nb_layer = 2))
        #print('output sortie :', output)


        loss = L.forward(output, target = train_target[j])
        loss_training = loss_training + loss
        
        #print('output entré dans backward :',output.size(), output)
        dl = L.backward(output, target = train_target[j])
        #print('output entré dans lin1 backward :', dl.size(), dl)
        #print('dl',dl)
        bf2 = lin2.backward(dl)
        #print('bf2', bf2.size(), bf2)
        bt1 = T1.backward(bf2)
        #print('bt1', bt1)
        bf1 = lin1.backward(bt1)
        #print('bf1',bf1)

        lr = 0.001 #>0.025 donne des meilleurs résultats (sinon oscille ou bloque)
        #print(lin1.param()[0][0])
        lin1.updateparam(lr)
        lin2.updateparam(lr)
        #print(lin1.param()[0][0])
    print('loss :', loss_training)

50

iteration 0
loss : tensor(275.0952)

iteration 1
loss : tensor(254.7361)

iteration 2
loss : tensor(253.7498)

iteration 3
loss : tensor(252.8886)

iteration 4
loss : tensor(252.0308)

iteration 5
loss : tensor(251.1161)

iteration 6
loss : tensor(250.0982)

iteration 7
loss : tensor(248.9326)

iteration 8
loss : tensor(247.5675)

iteration 9
loss : tensor(245.9406)

iteration 10
loss : tensor(243.9708)

iteration 11
loss : tensor(241.5573)

iteration 12
loss : tensor(238.5749)

iteration 13
loss : tensor(234.8779)

iteration 14
loss : tensor(230.3055)

iteration 15
loss : tensor(224.6931)

iteration 16
loss : tensor(217.8980)

iteration 17
loss : tensor(209.8297)

iteration 18
loss : tensor(200.4906)

iteration 19
loss : tensor(190.0112)

iteration 20
loss : tensor(178.6503)

iteration 21
loss : tensor(166.7588)

iteration 22
loss : tensor(154.7162)

iteration 23
loss : tensor(142.8907)

iteration 24
loss : tensor(131.6328)

iteration 25
loss : tensor(121.2895)

iteration 26
loss 

In [46]:
#Test après
errors_training = 0
for j in range(training_set.size(0)):
        f1 = lin1.forward(training_set[j].unsqueeze(0), nb_layer = 1)
        t1 = T1.forward(f1)
        output = (lin2.forward(t1, nb_layer = 2))

        pred = output.squeeze().max(0)[1].item() # 1 x 2 -> 2
        if train_target[j, pred].item() < 0.5: 
            #print(output, train_target[j])
            errors_training = errors_training + 1

print(' errors training: ', errors_training)

errors_testing = 0
for j in range(testing_set.size(0)):
    f1 = lin1.forward(testing_set[j].unsqueeze(0), nb_layer = 1)
    t1 = T1.forward(f1)
    output = (lin2.forward(t1, nb_layer = 2))

    pred = output.squeeze().max(0)[1].item() # 1 x 2 -> 2
    if test_target[j, pred] < 0.5: 
        #print(output, test_target[j])
        errors_testing = errors_testing + 1

print(' errors testing: ', errors_testing,)

 errors training:  47
 errors testing:  49


# SEQUENTIAL